<div style="font-size:18pt;padding-top:20px; text-align:center">Домашнее задание 4. <b>Рекомендательные системы и Spark MLlib</b> </div><hr>
<div style="text-align:right;">Куценко А. А <span style="font-style: italic;font-weight: bold;">(ftruf357ft@gmail.com)</span></div>

In [1]:
# Вариант
eu_position = 8
eu_position % 3

2

## Задание 2. Коллаборативная фильтрация

- Вариант 1. По схожести пользователей
- Вариант 2. По схожести объектов

Этапы:
1. Разделите данные с рейтингами на обучающее (train_init - 0.8) и тестовое подмножества (test - 0.2), определите среднее значение рейтинга в обучающем подмножестве и вычислите `rmse` для тестового подмножества, если для всех значений из test предсказывается среднее значение рейтинга
2. Реализуйте коллаборативную фильтрацию в соответствии с вариантом. Для определения схожести используйте train_init, для расчета `rmse` - test
3. Определите `rmse` для тестового подмножества

Environment setup

## 0. Подключение библиотек и создание Spark контекста

In [2]:
import os
import sys

os.environ["SPARK_HOME"]="/home/ubuntu/BigData/spark"
os.environ["PYSPARK_PYTHON"]="/home/ubuntu/ML/anaconda3/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"]="/home/ubuntu/ML/anaconda3/bin/python"

spark_home = os.environ.get("SPARK_HOME")
sys.path.insert(0, os.path.join(spark_home, "python"))
sys.path.insert(0, os.path.join(spark_home, "python/lib/py4j-0.10.7-src.zip"))

In [3]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [4]:
conf = pyspark.SparkConf() \
        .setAppName("moviewRecomApp") \
        .setMaster("local[16]") \
        .set("spark.executor.memory", "16g") \
        .set("spark.executor.core", "4") \
        .set("spark.driver.memory", "32g") \

In [5]:
spark = SparkSession.builder \
    .config(conf=conf) \
    .getOrCreate()

In [6]:
spark

## 1. Загрузка данных, разделение на обучающее и тестовое множество, расчет метрики базовой модели

In [7]:
#!sudo apt install tree
!tree dataset
!ls -lah dataset/ml-latest-small/ratings.csv
!ls -lah dataset/ml-latest/ratings.csv

dataset
├── ml-latest
│   ├── genome-scores.csv
│   ├── genome-tags.csv
│   ├── links.csv
│   ├── movies.csv
│   ├── ratings.csv
│   ├── README.txt
│   └── tags.csv
└── ml-latest-small
    ├── links.csv
    ├── movies.csv
    ├── ratings.csv
    ├── README.txt
    └── tags.csv

2 directories, 12 files
-rw-r--r-- 1 ubuntu ubuntu 2.4M Sep 26  2018 dataset/ml-latest-small/ratings.csv
-rw-rw-r-- 1 ubuntu ubuntu 891M Jul 20  2023 dataset/ml-latest/ratings.csv


Подгружаем в HDFS

In [8]:
!hdfs dfs -mkdir -p /data/recom/small
!hdfs dfs -mkdir -p /data/recom/large

In [9]:
!hdfs dfs -copyFromLocal /home/ubuntu/Documents/BigDataHWs/hw4/dataset/ml-latest-small/ratings.csv  /data/recom/small/ratings.csv
!hdfs dfs -copyFromLocal /home/ubuntu/Documents/BigDataHWs/hw4/dataset/ml-latest/ratings.csv  /data/recom/large/ratings.csv

copyFromLocal: `/data/recom/small/ratings.csv': File exists
copyFromLocal: `/data/recom/large/ratings.csv': File exists


In [10]:
!hdfs dfs -ls -h /data/recom/small
!hdfs dfs -ls -h /data/recom/large

Found 2 items
-rw-r--r--   1 ubuntu supergroup    482.8 K 2025-12-08 18:42 /data/recom/small/movies.csv
-rw-r--r--   1 ubuntu supergroup      2.4 M 2025-12-08 18:43 /data/recom/small/ratings.csv
Found 2 items
-rw-r--r--   1 ubuntu supergroup      4.0 M 2025-12-08 19:01 /data/recom/large/movies.csv
-rw-r--r--   1 ubuntu supergroup    890.6 M 2025-12-08 19:01 /data/recom/large/ratings.csv


In [11]:
# Доступ к данным из HDFS для небольшего набора данных
ratings_data_small_path = "hdfs:///data/recom/small/ratings.csv"

# Доступ к данным из HDFS для полного набора данных
ratings_data_full_path = "hdfs:///data/recom/large/ratings.csv"

RATINGS_FILE = ratings_data_full_path

In [12]:
df_ratings = spark.read.load(RATINGS_FILE,
                          format="csv",
                          header="true",
                          inferSchema="true",
                          sep=",")

df_ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [13]:
n_partitions = df_ratings.rdd.getNumPartitions()
print(n_partitions)

16


In [14]:
df_ratings = df_ratings.repartition(n_partitions * 4)
df_ratings.rdd.getNumPartitions()

64

Делим данные на обучающее и тестовове подмножество

In [15]:
df_train, df_test = df_ratings.randomSplit([0.8, 0.2], seed=12)
df_train.persist().count(), df_test.persist().count()

(27065950, 6766212)

In [16]:
df_train.rdd.getNumPartitions()

64

Средний рейтинг фильмов

In [17]:
mean_movie_rating = df_train.select(F.mean("rating").alias("meanRating")).collect()[0]["meanRating"]
mean_movie_rating

3.542462300418053

Тестирование базовой модели. Всегда предсказываем среднее

In [18]:
df_test.show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     5|   3185|   4.0|1029389203|
|    10|  91529|   4.5|1430667372|
|    14|    260|   4.5|1311532965|
|    21|    173|   1.0|1172766335|
|    22| 106100|   5.0|1536154114|
+------+-------+------+----------+
only showing top 5 rows



In [19]:
df_test_pred_bl = df_test.withColumn("prediction", F.lit(mean_movie_rating))
df_test_pred_bl.show(5)

+------+-------+------+----------+-----------------+
|userId|movieId|rating| timestamp|       prediction|
+------+-------+------+----------+-----------------+
|     5|   3185|   4.0|1029389203|3.542462300418053|
|    10|  91529|   4.5|1430667372|3.542462300418053|
|    14|    260|   4.5|1311532965|3.542462300418053|
|    21|    173|   1.0|1172766335|3.542462300418053|
|    22| 106100|   5.0|1536154114|3.542462300418053|
+------+-------+------+----------+-----------------+
only showing top 5 rows



Считаем RMSE на тестовом наборе

In [20]:
from pyspark.ml.evaluation import RegressionEvaluator

In [21]:
eval_rmse = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
eval_rmse.evaluate(df_test_pred_bl)

1.0638966156054195

## 2. Коллаборативная фильтрация на основе схожести объектов

In [22]:
spark.catalog.clearCache()

In [23]:
sys.path.insert(0, "/home/ubuntu/Documents/BigDataHWs/hw4/lib/recommend")

In [24]:
from itemrecom import ItemBasedRecommend

In [25]:
item_based = ItemBasedRecommend(spark)

**Обучение**

In [26]:
item_based.train(df_train, user_column_name="userId", item_column_name="movieId")

**Предсказание**

In [27]:
USER = 123
MOVIE_ID = 1234

а) Предсказываем рейтинг одного фильма для одного пользователя

In [28]:
item_based.predict_single(USER, MOVIE_ID)

4.674583682536866

б) Список рекомендаций для пользователя

In [29]:
TOP_N_RATINGS = 20

In [30]:
df_recom = item_based.recommend(user_ids=[USER], top_N_ratings=TOP_N_RATINGS, grouped=False, partition_num=128)
df_recom.show()

+----+-----------------+----+
|item|      rating_pred|user|
+----+-----------------+----+
| 728|5.000000000000001| 123|
| 905|5.000000000000001| 123|
| 931|5.000000000000001| 123|
| 948|5.000000000000001| 123|
| 971|5.000000000000001| 123|
|1237|5.000000000000001| 123|
|1401|5.000000000000001| 123|
|1939|5.000000000000001| 123|
|2142|5.000000000000001| 123|
|2203|5.000000000000001| 123|
|2204|5.000000000000001| 123|
|2772|5.000000000000001| 123|
|2922|5.000000000000001| 123|
|3089|5.000000000000001| 123|
|3549|5.000000000000001| 123|
|4406|5.000000000000001| 123|
|4498|5.000000000000001| 123|
|4591|5.000000000000001| 123|
| 949|5.000000000000001| 123|
|5008|5.000000000000001| 123|
+----+-----------------+----+



## 3. Тестирование

In [31]:
# Все уникальные пользователи в тестовом множестве
U = {el["userId"] for el in df_test[[F.col("userId")]].distinct().collect()}

print("Уникальных пользователей: ", len(U))

Уникальных пользователей:  305858


In [32]:
df_test_predicted = item_based.predict(df_test, user_column_name="userId", item_column_name="movieId")
df_test_predicted.show()

+------+----+------------------+
|  user|item|       rating_pred|
+------+----+------------------+
|321627|1127| 4.250301020732027|
|190539| 519|2.9732897467170822|
|266231| 379| 2.992421703265427|
|280494|1676| 3.947523170882441|
|104306|1544|3.7224053484299247|
| 42068|1690| 3.139618461925116|
|158939|1127|  3.91853013198041|
|244499|2455|3.1866146602406884|
|323858|1544| 2.749598046925327|
| 68214|2916|1.9463080822468228|
|146375| 379|3.4999999999999996|
|133664|1590| 3.871693654940541|
|270155|3527|  2.97993385255655|
| 77219|1255|  4.02487009301907|
|195299|2455| 3.250749371198191|
|188407| 379|3.5000000000000004|
|284393|1573|1.2705593827044346|
|292083|1391|3.5859818561238916|
|205325|2985|3.8602270560144682|
|309101| 849|3.3595999951504623|
+------+----+------------------+
only showing top 20 rows



In [33]:
from pyspark.storagelevel import StorageLevel

In [34]:
df_true_pred = df_test.join(df_test_predicted, on=[(F.col("userId")==F.col("user")), 
                                                   (F.col("movieId")==F.col("item"))])
df_true_pred.persist(StorageLevel.MEMORY_ONLY).count()
df_true_pred.show(5)

+------+-------+------+----------+----+----+------------------+
|userId|movieId|rating| timestamp|user|item|       rating_pred|
+------+-------+------+----------+----+----+------------------+
|    21|   3686|   5.0|1173150744|  21|3686|               2.0|
|    24|    302|   4.5|1062938525|  24| 302|               4.0|
|    24|   3624|   3.5|1062939808|  24|3624|3.6132913846775274|
|    53|   2997|   4.0|1134596942|  53|2997| 4.034656831669875|
|    53|   7361|   4.5|1134596041|  53|7361| 3.840828356121518|
+------+-------+------+----------+----+----+------------------+
only showing top 5 rows



In [35]:
df_true_pred.rdd.getNumPartitions()

200

In [36]:
eval_rmse = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="rating_pred")
eval_rmse.evaluate(df_true_pred)

0.8443230507480239

In [37]:
spark.stop()